<a href="https://colab.research.google.com/github/RajuGuguloth/DL-Assignment-2/blob/main/DL_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup and Imports**

In [ ]:
 # 📦 Required installations
!pip install -q wandb

# 📚 Importing essential libraries
import os
import random
import zipfile
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms, utils

import wandb


# **W&B Login and Device Setup**

In [ ]:
# 🧑‍💻 W&B Login
wandb.login()

# 🖥️ Device Setup (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# **Data Preparation**

In [ ]:
# 📂 Dataset Path Setup
data_dir = 'path_to_your_data_directory'

# 🔄 Image Transformations for Data Augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resizing the image to a standard size
    transforms.ToTensor(),  # Converting images to tensor format
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizing based on ImageNet statistics
])

# 📂 Loading Data with ImageFolder
dataset = ImageFolder(root=data_dir, transform=transform)

# 🔀 Splitting Dataset into Training and Validation Sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 🧑‍🏫 DataLoader for Batch Loading
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Training set size: {len(train_dataset)} | Validation set size: {len(val_dataset)}")


# **CNN Model Definition**

In [ ]:
# 🏗️ Defining the CNN Architecture
class CNN(nn.Module):
    def __init__(self, num_classes=1000):
        super(CNN, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        # Fully Connected Layers
        self.fc1 = nn.Linear(128*16*16, 512)  # Flattening the image after convolutions
        self.fc2 = nn.Linear(512, num_classes)

        # Dropout Layer for regularization
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Forward pass through the network
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # Flatten the output from convolutional layers
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))  # Fully connected layer
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)  # Final output layer
        return x


# **Loss Function and Optimizer Setup**

In [ ]:
# ⚖️ Loss Function: Cross-Entropy Loss
criterion = nn.CrossEntropyLoss()

# 🛠️ Optimizer: Adam with weight decay (L2 regularization)
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
